### Objective

In this notebook, I implemented Unity Catalog governance concepts to understand how data is organized, secured, and shared in real-world enterprise data platforms using Databricks.

### Catalog & Schema Setup




Created a catalog named ecommerce

Created three schemas to align with Medallion Architecture:

bronze → raw data

silver → cleaned data

gold → business-ready data

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ecommerce;
USE CATALOG ecommerce;

CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;

### Register Delta Tables
Created managed Delta tables 

Registered tables in each layer:
- bronze.events
- silver.events
- gold.products



In [0]:
%sql
USE CATALOG ecommerce;

-- BRONZE
CREATE TABLE IF NOT EXISTS bronze.events AS
SELECT
  CAST(1 AS INT)      AS event_id,
  'view'              AS event_type,
  current_timestamp() AS event_ts
UNION ALL
SELECT
  2, 'purchase', current_timestamp();

-- SILVER
CREATE TABLE IF NOT EXISTS silver.events AS
SELECT * FROM bronze.events;

-- GOLD
CREATE TABLE IF NOT EXISTS gold.products AS
SELECT
  'Product A' AS product_name,
  12000       AS revenue,
  0.12        AS conversion_rate,
  25          AS purchases
UNION ALL
SELECT
  'Product B', 8000, 0.09, 8;


num_affected_rows,num_inserted_rows


Verified data using SELECT queries

In [0]:
%sql
SELECT * FROM bronze.events;
SELECT * FROM silver.events;
SELECT * FROM gold.products;


product_name,revenue,conversion_rate,purchases
Product A,12000,0.12,25
Product B,8000,0.09,8


### Access Control (GRANT)

Checked existing permissions using SHOW GRANTS

In [0]:
%sql
SHOW GRANTS ON TABLE gold.products;


Principal,ActionType,ObjectType,ObjectKey
account users,SELECT,TABLE,ecommerce.gold.products


Granted SELECT-only access on gold.products to account users


In [0]:
%sql
GRANT SELECT ON TABLE gold.products TO `account users`;


Verified permissions using SHOW GRANTS

In [0]:
%sql
SHOW GRANTS ON TABLE gold.products;

Principal,ActionType,ObjectType,ObjectKey
account users,SELECT,TABLE,ecommerce.gold.products


### Controlled Access Using Views
Created a view gold.top_products

Applied business logic: purchases > 10


In [0]:
%sql
USE CATALOG ecommerce;

CREATE OR REPLACE VIEW gold.top_products AS
SELECT
  product_name,
  revenue,
  conversion_rate
FROM gold.products
WHERE purchases > 10
ORDER BY revenue DESC
LIMIT 100;



Showing: product_name, revenue and conversion_rate

In [0]:
%sql
SELECT * FROM gold.top_products;


product_name,revenue,conversion_rate
Product A,12000,0.12


Granted SELECT access on the view, not the base table

In [0]:
%sql
GRANT SELECT ON VIEW gold.top_products TO `account users`;


In [0]:
%sql
SHOW GRANTS ON VIEW gold.top_products;


Principal,ActionType,ObjectType,ObjectKey
account users,SELECT,TABLE,ecommerce.gold.top_products
